[Description]: in the notebook facerecognization is testing with base classifiers using landmarks detected from dlib


In [64]:
import os
from custom_model.basic_landmark_classifier import LandmarkClassifier
import PIL.Image
import cv2
import dlib
import numpy as np
from imutils import face_utils
from pathlib import Path
import PIL
import pandas as pd
import torch
class FRBase:
    def __init__(self,landmark_weight_path,fr_weight_path,classifier_weight_path,id_name_csv_path) :
        names_df=pd.read_csv(id_name_csv_path,index_col=0)
        self.ids_name=names_df[["name","id"]].groupby(["name","id"]).mean().reset_index()

        
        self.landmark_weight_path=landmark_weight_path
        self.fr_weight_path=fr_weight_path
        self.classifier_weight_path=Path(classifier_weight_path)
        
        self.load_pretrained_weight()


    def get_class_name(self,pred_id):
        name=self.ids_name[self.ids_name["id"]==pred_id]["name"].values[-1]
        return name.split("_")[-1]
    def load_pretrained_weight(self):
        try:
            self.face_detector = dlib.get_frontal_face_detector()
            self.pose_predictor_68_point = dlib.shape_predictor(self.landmark_weight_path)
            print(f"pose_predictor_68_point weight loaded successfully")
        except  Exception as e:
            print(f"pose_predictor_68_point loading weight exception occur {e}")
        
        try:
            self.classifier = LandmarkClassifier(136,25)

            print(f"classifier weight loaded successfully")
        except  Exception as e:
            print(f"classifier loading weight exception occur {e}")
        

    def set_data_paths(self,all_names):
        data_paths={}
        for name in all_names:
            new_path=self.root.joinpath(name)
            images_path=list(new_path.rglob("*.jpg")) + list(new_path.rglob("*.png"))
            data_paths[name]=images_path
        return data_paths
    def get_data(self,):
        return self.data
    def get_gray_img(self,image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        return gray_image

    def get_transform_image(self,image):
        #assert len(image.shape) == 3, f"[ERROR] The input image does not have three dimensions. Current shape: {image.shape}"
        if len(image.shape)!=2:
            if image.shape[2] == 4:
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
            elif image.shape[2] == 3:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            elif image.shape[2] == 1:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            else:
                assert False,f"[ERROR] Unsupported number of channels: {image.shape[2]}." 
            
            if image.dtype != np.uint8:
                image = image.astype(np.uint8)
            
            image=self.get_gray_img(image)
        return image
    def make_landmark_list(self,face_locations,image):
        landmarks_list = []
        for face_location in face_locations:
            try: 
                shape = self.pose_predictor_68_point(image, face_location)
                #face_encodings_list.append(np.array(face_encoder.compute_face_descriptor(image, shape, num_jitters=1)))
                shape = face_utils.shape_to_np(shape)
                landmarks_list.append(shape) 
            except Exception as e:
                print(f"exception:{e}")
        return landmarks_list     
    def get_face_landmarks(self,image)->list:
        
    
            

        # Ensure the image is in the correct format

        gray_image=self.get_transform_image(image)
        
        face_locations=list()
        try:
            face_locations =self.face_detector(gray_image, 1)
        except Exception as e:
            print(f"[ERROR] Failed to detect faces. Error: {e}")
            return []

        if len(face_locations) == 0:
            
            return []

        return  self.make_landmark_list(face_locations,image)
    def save_landmarks(self, landmarks, img_path)->None:
        user_directory = self.directory_to_save / Path(img_path).parent.name
        if not user_directory.exists():
            user_directory.mkdir(parents=True, exist_ok=True)
        
        # Save landmarks as .npy file
        save_path = user_directory / (Path(img_path).stem + "_landmarks.npy")
        np.save(save_path, landmarks)
        print(f"[INFO] Landmarks saved for {img_path} at {save_path}")
    def detect_landmarks_vid(self, video_path,resize_width=460)->None:
        cam = cv2.VideoCapture(str(video_path))
        cv2.namedWindow("Capture Face")
        
        while True:
                ret, frame = cam.read()
                if not ret:
                    break                
                # Resize the frame to the specified width while maintaining aspect ratio
                height, width = frame.shape[:2]
                aspect_ratio = height / width
                new_height = int(resize_width * aspect_ratio)
                frame_resized = cv2.resize(frame, (resize_width, new_height))

                landmark_list = self.get_face_landmarks(frame_resized)
                if len(landmark_list)!=0:
                    
                    landmark_list_tensored=torch.tensor(landmark_list,dtype=torch.float32).unsqueeze(1)
                   
                    probs,logits = self.classifier(landmark_list_tensored)
                    for i,shape in enumerate(landmark_list):
                        #self.put_landmarks(frame_resized, shape)

                        x_min, y_min = np.min(shape, axis=0)
                        x_max, y_max = np.max(shape, axis=0)

                        
                    
                        #_, predicted = torch.max(logits, 1)
                        #print(f"predicted:{predicted}")
                        at_prob=probs[i]
                        _, indices = torch.max(probs, dim=0)
                        
                        index=indices[i].item()
                        prob=at_prob[index]
                        predicted=-1
                        #print(f"prob:{prob}")
                        if prob>0.5:
                            text_color = (0, 255, 0)
                            cls_name=self.get_class_name(index)
                        elif prob<0.2:
                            cls_name="not in database"
                            text_color=(0, 0, 255)
                        else:
                            cls_name="recognizing..."
                            text_color = (255, 255, 255)
                            
                        # Draw the rectangle using the min/max x and y coordinates
                        cv2.rectangle(frame_resized, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                        # Adjusted parameters for better text visibility
                        cv2.putText(frame_resized, cls_name, (x_min + 2, y_max - 2),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, text_color,1, cv2.LINE_AA)  # White text, thicker, anti-aliased

                        # Left Side: x_min
                        # Top Side: y_min
                        # Right Side: x_max
                        # Bottom Side: y_max


                cv2.imshow("Capture Face", frame_resized)
                
                k = cv2.waitKey(1)
                if k % 256 == 27 :  # ESC or 20 images collected
                    break

        cam.release()
        cv2.destroyAllWindows()

    def put_landmarks(self, frame_resized, shape):
        for (x, y) in shape:
            cv2.circle(frame_resized, (x, y), 1, (255, 0, 255), -1)



        
         
        
obj=FRBase(
    "pretrained_model/shape_predictor_68_face_landmarks.dat",
    "pretrained_model/dlib_face_recognition_resnet_model_v1.dat",
    "./weights/weight_basic.pth",
    "./weights/name_id.csv"
    )



pose_predictor_68_point weight loaded successfully
classifier weight loaded successfully


In [66]:
obj.detect_landmarks_vid("./videos/video.mp4",600)